In [10]:
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 

import multiprocessing
import mkl
import time
mkl.set_num_threads(12)
mkl.get_max_threads()

12

In [17]:
h = 1.0
V = 1.0
alpha = 1 
length = [8, 8]
name = 'h={}V={}l={}'.format(h, V, length)
kernel = 1
n_chains = 1
n_discards = 1
n_samples = 1e5

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)

4


In [20]:
mkl.set_num_threads(1)
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70, kernel = 1, n_jobs=12)
sa_mul.reset()
print(sa_mul.sa_list[0]._state)
samples_state = sa_mul.generate_samples(10**5)
samples_state = samples_state.reshape(-1, samples_state.shape[-1])

number of core : 12
0.020051240921020508 for metropolis
[[-1. -1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.
   1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.
   1.  1.  1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1. -1. -1.
  -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1. -1.
  -1. -1. -1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.
   1.  1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1.  1.  1.
  -1.  1.]]
14.568799257278442 for metropolis
14.60584378242492714.615193605422974 for metropolis 
for metropolis
14.640964269638062 for metropolis
14.673559665679932 for metropolis
14.6926908493042 for metropolis
14.750976324081421 for metropolis
14.808945417404175 for metropolis
14.848471641540527 for metropolis
15.29661750793457 for metropolis
15.460580825805664 for metropolis
16.58040690422058 for metropolis


In [26]:
mkl.set_num_threads(12)

1

In [27]:
%timeit ma.log_val(samples_state)

182 ms ± 428 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit r = samples_state.dot(ma._w)

112 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
out = np.empty(samples_state.shape[0], dtype=np.complex128)

In [24]:
%timeit nk.machine.rbm._log_cosh_sum(r, out)

134 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
